In [8]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import pandas as pd
import matplotlib.pyplot as plt
import os

In [9]:
dataset = pd.read_pickle("./new_dataset.pkl")
dataset

,ageRange,path
18640,0,./UTKFace/26_1_3_20170117153712926.jpg.chip.jpg
8878,0,./UTKFace/35_1_0_20170117092205487.jpg.chip.jpg
7495,0,./UTKFace/30_1_2_20170116192643744.jpg.chip.jpg
3851,0,./UTKFace/31_1_4_20170112235504601.jpg.chip.jpg
3407,0,./UTKFace/36_0_1_20170120221302965.jpg.chip.jpg
...,...,...
18557,3,./UTKFace/85_1_2_20170112204748587.jpg.chip.jpg
14014,3,./UTKFace/85_1_0_20170110182918046.jpg.chip.jpg
2277,3,./UTKFace/80_1_1_20170116153903211.jpg.chip.jpg
18416,3,./UTKFace/85_0_0_20170111210559755.jpg.chip.jpg


In [10]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

batch_size = 32
epochs = 50
input_shape = (256, 256, 3)

# 이미지 증강 세팅
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)



# 학습 데이터셋 생성
train_generator = train_datagen.flow_from_dataframe(
    train_data,
    x_col='path',
    y_col='ageRange',
    target_size=input_shape[:2],
    batch_size = batch_size,
    class_mode='categorical'
)

# 검증 데이터셋 생성
test_generator = test_datagen.flow_from_dataframe(
    test_data,
    x_col='path',
    y_col='ageRange',
    target_size=input_shape[:2],
    batch_size = batch_size,
    class_mode='categorical'
)

Found 9600 validated image filenames belonging to 4 classes.
Found 2400 validated image filenames belonging to 4 classes.


In [133]:
from tensorflow import keras

from tensorflow.keras.preprocessing import image
import numpy as np

model = keras.models.load_model('./model/ResNet50V2/best.h5')

In [134]:
# 예측할 이미지 불러오기
img_path = './test_img/고.jpeg'
img = image.load_img(img_path, target_size=(256, 256))

# 이미지 전처리
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.

# 모델 예측
prediction = model.predict(img_array)

predClass = np.argmax(prediction)

In [135]:
print(prediction)

[[6.3395309e-01 2.5249940e-01 1.1354747e-01 9.8780120e-08]]


In [136]:
print(predClass)

0


In [137]:
'''
0: 2~30대
1: 4~50대
2: 6~70대
3: 80대 이상
'''

if predClass == 0:
    print("20~39세")
elif predClass == 1:
    print("40~59세")
elif predClass == 2:
    print("60~79세")
else:
    print("80세 이상")

20~39세
